# Lab 1: Configuração Interativa do Banco de Dados (PostgreSQL)

Este notebook executa a configuração do banco de dados PostgreSQL de forma interativa. As células de código a seguir irão se conectar ao contêiner do banco de dados e executar os scripts de criação de tabelas (DDL) e inserção de dados (DML) usando Python.

## 1. Inicializar os bancos de dados

Etapa obrigatória, pois todos os serviços iniciam desligados para que o MYBinder consiga inicializar o Container.

In [ ]:
!/usr/local/bin/entrypoint.sh

+ mkdir -p /home/jovyan/logs
+ echo 'Starting services...'
Starting services...
+ nohup redis-server
+ sudo /usr/sbin/sshd -f /etc/ssh/sshd_config
+ echo 'Waiting for SSH port 8822 to open...'
Waiting for SSH port 8822 to open...
+ nohup /home/jovyan/resources/code-server-3.4.1/bin/code-server
+ ss -tuln
+ grep -q :8822
+ echo 'SSH port 8822 is open.'
SSH port 8822 is open.
+ echo 'Waiting for SSH to be ready for authentication...'
Waiting for SSH to be ready for authentication...
+ ssh -o StrictHostKeyChecking=no -p 8822 jovyan@localhost exit
+ echo 'SSH is ready for authentication.'
SSH is ready for authentication.
+ sudo pg_ctlcluster 14 main start
Cluster is already running.
+ sleep 1
+ echo 'Waiting for PostgreSQL to be ready...'
Waiting for PostgreSQL to be ready...
+ pg_isready -h localhost -p 5432 -q
+ echo 'PostgreSQL is ready.'
PostgreSQL is ready.
+ cd /tmp
+ sudo -u postgres psql -c 'CREATE DATABASE postgres;'
+ echo 'Database postgres already exists.'
Database postgres alr

## 2. Conexão e Execução dos Scripts

A célula abaixo contém todo o processo:
1.  Importa as bibliotecas necessárias.
2.  Define as queries DDL e DML.
3.  Estabelece uma conexão com o contêiner `petshop_db` (o nome do serviço no `docker-compose`).
4.  Cria um cursor e executa as queries.
5.  Confirma as transações e fecha a conexão.

In [ ]:
import psycopg2
import os

# As credenciais e o host são baseados no arquivo docker-compose.txt
DB_HOST = "localhost" # Nome do serviço no Docker Compose
DB_NAME = "postgres"
DB_USER = "postgres"
DB_USER_PWD = "postgres"

DDL_SCRIPT = """
CREATE TABLE IF NOT EXISTS tutor (
    tutor_id SERIAL PRIMARY KEY,
    nome VARCHAR(255) NOT NULL,
    email VARCHAR(255) UNIQUE NOT NULL
);

CREATE TABLE IF NOT EXISTS pet (
    pet_id SERIAL PRIMARY KEY,
    tutor_id INTEGER NOT NULL REFERENCES tutor(tutor_id),
    nome VARCHAR(100) NOT NULL,
    especie VARCHAR(50) NOT NULL,
    tipo_pelo VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS agendamento (
    agendamento_id SERIAL PRIMARY KEY,
    pet_id INTEGER NOT NULL REFERENCES pet(pet_id),
    tipo_servico VARCHAR(100) NOT NULL,
    data_agendamento TIMESTAMP WITH TIME ZONE NOT NULL,
    status VARCHAR(50) NOT NULL
);

CREATE TABLE IF NOT EXISTS produto (
    produto_id SERIAL PRIMARY KEY,
    nome_produto VARCHAR(255) NOT NULL,
    categoria VARCHAR(100),
    preco NUMERIC(10, 2) NOT NULL
);

CREATE TABLE IF NOT EXISTS compra (
    compra_id SERIAL PRIMARY KEY,
    tutor_id INTEGER NOT NULL REFERENCES tutor(tutor_id),
    produto_id INTEGER NOT NULL REFERENCES produto(produto_id),
    data_compra DATE NOT NULL,
    quantidade INTEGER NOT NULL
);
"""

DML_SCRIPT = """
INSERT INTO tutor (nome, email) VALUES
('Ana Carolina', 'ana.carolina@email.com'),
('Bruno Martins', 'bruno.martins@email.com')
ON CONFLICT (email) DO NOTHING;

INSERT INTO pet (tutor_id, nome, especie, tipo_pelo) VALUES
(1, 'Bidu', 'Cão', 'Longo'),
(1, 'Luna', 'Gato', 'Curto'),
(2, 'Thor', 'Cão', 'Longo');

INSERT INTO agendamento (pet_id, tipo_servico, data_agendamento, status) VALUES
(1, 'Banho e Tosa Completa', '2025-04-10 14:00:00', 'Realizado'),
(1, 'Banho e Tosa Completa', '2025-05-11 14:00:00', 'Realizado'),
(1, 'Banho e Tosa Completa', '2025-06-12 14:00:00', 'Realizado'),
(3, 'Banho', '2025-07-01 11:00:00', 'Realizado');

INSERT INTO produto (nome_produto, categoria, preco) VALUES
('Ração para Cães de Pelo Longo', 'Alimentação', 75.50),
('Shampoo Hipoalergênico para Cães', 'Higiene', 30.00);

INSERT INTO compra (tutor_id, produto_id, data_compra, quantidade) VALUES
(1, 1, '2025-06-15', 1), (1, 2, '2025-06-15', 1), (2, 1, '2025-06-22', 1);
"""

try:
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_USER_PWD)
    cur = conn.cursor()
    
    print("Executando script DDL (criação de tabelas)...")
    cur.execute(DDL_SCRIPT)
    print("DDL executado com sucesso.")
    
    print("Executando script DML (inserção de dados)...")
    # Limpa as tabelas antes de inserir para garantir que o script seja idempotente
    cur.execute("TRUNCATE TABLE agendamento, compra, pet, tutor, produto RESTART IDENTITY CASCADE;")
    cur.execute(DML_SCRIPT)
    print("DML executado com sucesso.")
    
    conn.commit()
    print("Transação commitada.")
    
except psycopg2.OperationalError as e:
    print(f"Erro de conexão: {e}")
    print("Verifique se os contêineres Docker estão em execução ('docker-compose up -d') e se o nome do host do banco de dados ('{DB_HOST}') está correto.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")
finally:
    if 'conn' in locals() and conn is not None:
        cur.close()
        conn.close()
        print("Conexão fechada.")

## 3. Verificação

Execute a célula abaixo para se conectar novamente e fazer uma consulta `SELECT` para verificar se os dados foram inseridos corretamente na tabela `agendamento`.

In [ ]:
try:
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_USER_PWD)
    cur = conn.cursor()
    
    cur.execute("SELECT * FROM agendamento;")
    rows = cur.fetchall()
    
    print("Agendamentos encontrados:")
    for row in rows:
        print(row)
        
except Exception as e:
    print(f"Ocorreu um erro: {e}")
finally:
    if 'conn' in locals() and conn is not None:
        cur.close()
        conn.close()